# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [1]:
import import_ipynb
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 50
n_product = 5
n_consumer = 100
market_variance = 0.01 #amount of variation between markets 
endogenous_noise = 0.01 #how noise to generate price 

#structural parameters 
true_alpha = 3 #decrease with utility 
true_beta = 13 #increase with utility
true_sigma_alpha = 0.7 
true_sigma_beta = 0.3
market_variance = 0.01 #amount of variation between markets 

In [2]:
all_markets = []
for market in range(n_market):
    x = np.random.uniform(0, 1, n_product)
    instrument = np.random.uniform(0, 10, n_product)

    # generate market shares and price variables endogenously 
    price = f.price_gen(x, instrument, endogenous_noise)
    inner_utility = f.utility_gen([true_alpha, true_beta], [true_sigma_alpha, true_sigma_beta], x, price)
    inner_market_utility = f.market_gen(inner_utility, market_variance)
    inner_prob_choice = f.choice_prob(pd.DataFrame(inner_market_utility))
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=0)/n_consumer

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'x': x, 
        'instrument': instrument, 
        'price': price,
        'share': outer_market_share, 
        'share_not_buy': 1 - sum(outer_market_share)
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets[:50])

export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_data.csv', index=False)

[   market_id  product_id         x  instrument     price     share  \
0          0           0  0.344071    3.793929  2.424128  0.004141   
1          0           1  0.102991    8.484313  5.231687  0.000001   
2          0           2  0.820901    5.185154  3.477507  0.184452   
3          0           3  0.422727    1.947232  1.342893  0.214451   
4          0           4  0.326673    1.591563  1.120880  0.118687   

   share_not_buy  
0       0.478267  
1       0.478267  
2       0.478267  
3       0.478267  
4       0.478267  ,    market_id  product_id         x  instrument     price     share  \
0          1           0  0.459961    7.839532  4.749005  0.000002   
1          1           1  0.336098    2.282543  1.477215  0.000533   
2          1           2  0.414200    1.252176  0.891550  0.007440   
3          1           3  0.734965    0.612654  0.644862  0.989069   
4          1           4  0.345110    5.165692  3.143989  0.000012   

   share_not_buy  
0       0.002943  
1   